In [1]:
# Initial imports
from alpaca_trade_api.rest import REST, TimeFrame
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import date
load_dotenv()
import requests
import pandas as pd
import numpy as np
import getpass
import os
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm

In [2]:
from MACDBacktester import MACDBacktester as MACD

In [3]:
# API Info for fetching data, portfolio, etc. from Alpaca
BASE_URL = "https://paper-api.alpaca.markets"

if os.getenv("APCA_API_KEY_ID") is not None:
    ALPACA_API_KEY = os.getenv("APCA_API_KEY_ID")
else:

    ALPACA_API_KEY = getpass.getpass('Type your alpaca api key here, it is safe from viewing.')

if os.getenv("ALPACA_SECRET_KEY") is not None:
    ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
else:

    ALPACA_SECRET_KEY = getpass.getpass('Type your secret alpaca api key here, it is safe from viewing.')

# Instantiate REST API Connection - Account
api_account = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, raw_data=True,base_url=BASE_URL, api_version='v2')

# Instantiate REST API Connection - Trade
api_trade = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, raw_data=False, base_url=BASE_URL, api_version='v2')

In [4]:
# Get the start date and end dates
start = "2009-01-03" # Bitcoin's genesis block date
end = "2022-03-19"

In [5]:
df = api_trade.get_crypto_bars("BTCUSD", TimeFrame.Day, start, end, exchanges=['CBSE']).df

In [6]:
df.tail()

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2022-03-15 05:00:00+00:00,CBSE,38792.35,41717.67,38135.00,39130.30,18289.667602,752620,39457.811868
2022-03-16 05:00:00+00:00,CBSE,39129.96,41498.99,38988.66,41364.35,19119.068627,906849,40539.666987
2022-03-17 05:00:00+00:00,CBSE,41360.67,41486.00,40261.58,40646.56,15356.127485,677071,40843.247963
2022-03-18 05:00:00+00:00,CBSE,40645.27,42396.04,40130.00,41824.46,16287.481907,642697,41261.104606
2022-03-19 05:00:00+00:00,CBSE,41823.30,42109.69,41560.49,42033.47,3033.553965,156823,41817.138905


In [7]:
df["returns"] = np.log(df['close']).diff().dropna()

In [8]:
df = df.dropna()

In [9]:
display(df.head())
display(df.tail())

,exchange,open,high,low,close,volume,trade_count,vwap,returns
timestamp,,,,,,,,,
2015-01-08 06:00:00+00:00,CBSE,297.08,314.91,265.50,288.99,3.930000,393,289.251781,0.048022
2015-01-12 06:00:00+00:00,CBSE,260.00,260.00,260.00,260.00,1.000000,1,260.000000,-0.105710
2015-01-13 06:00:00+00:00,CBSE,200.00,221.00,200.00,219.63,0.030000,3,213.543333,-0.168737
2015-01-14 06:00:00+00:00,CBSE,220.00,220.00,109.87,120.00,11.274638,1115,155.605993,-0.604453
2015-01-15 06:00:00+00:00,CBSE,199.00,224.00,199.00,204.22,1.775121,165,206.555271,0.531706


,exchange,open,high,low,close,volume,trade_count,vwap,returns
timestamp,,,,,,,,,
2022-03-15 05:00:00+00:00,CBSE,38792.35,41717.67,38135.00,39130.30,18289.667602,752620,39457.811868,0.008630
2022-03-16 05:00:00+00:00,CBSE,39129.96,41498.99,38988.66,41364.35,19119.068627,906849,40539.666987,0.055522
2022-03-17 05:00:00+00:00,CBSE,41360.67,41486.00,40261.58,40646.56,15356.127485,677071,40843.247963,-0.017505
2022-03-18 05:00:00+00:00,CBSE,40645.27,42396.04,40130.00,41824.46,16287.481907,642697,41261.104606,0.028567
2022-03-19 05:00:00+00:00,CBSE,41823.30,42109.69,41560.49,42033.47,3033.553965,156823,41817.138905,0.004985


In [10]:
# The independent variable, or the variable used to predict the dependent variable is denoted as x. 
# The dependent variable, or the outcome/output, is denoted as y
x = df[['vwap','trade_count']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
Y = df['returns'] # outcome id like to predict

In [11]:
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(x, Y)

LinearRegression()

In [12]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 0.0018579372707546618
Coefficients: 
 [ 1.38030266e-07 -2.34608405e-08]


In [13]:
# prediction with sklearn
New_vwap = 41817.138905
New_trade_count = 156823
print ('Predicted returns: \n', regr.predict([[New_vwap, New_trade_count]]))

Predicted returns: 
 [0.00395077]


C:\Users\Bum Rush\.conda\envs\algotrading\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


In [14]:
# with statsmodels
X = sm.add_constant(x) # adding a constant
 
model = sm.OLS(Y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                returns   R-squared (uncentered):                   0.003
Model:                            OLS   Adj. R-squared (uncentered):              0.002
Method:                 Least Squares   F-statistic:                              4.088
Date:                Sat, 19 Mar 2022   Prob (F-statistic):                      0.0169
Time:                        11:56:47   Log-Likelihood:                          4457.7
No. Observations:                2624   AIC:                                     -8911.
Df Residuals:                    2622   BIC:                                     -8900.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

C:\Users\Bum Rush\.conda\envs\algotrading\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
